In [ ]:
%matplotlib inline
from urllib import request, parse
from bs4 import BeautifulSoup
from pymongo import MongoClient
from bson import ObjectId
import numpy as np
import datetime
import ssl
import re
import time
import json
import random

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context  # 全局取消网页安全验证
client = MongoClient('localhost', 27017)
db = client.moniter_crowdfunding
project = db.projects
f_project = db.failure_projects
p_founder = db.founders

In [13]:
class Single_proj_craw:

    def __init__(self, p_id, count_inqury=0):
        self.User_Agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:60.0) Gecko/20100101 Firefox/60.0'
        self.Host = 'z.jd.com'
        self.p_id = p_id
        self.count_inqury = count_inqury  # 访问次数
        self.p_d = re.compile('\d+')
        self.craw_time = datetime.datetime.now()
        self.category = False

        url_1 = 'http://z.jd.com/project/details/%s.html' % self.p_id
        req_1 = request.Request(url_1)
        req_1.add_header('User-Agent', self.User_Agent)
        req_1.add_header('Host', self.Host)
        with request.urlopen(req_1) as f_1:
            self.isDirected = self.p_id in f_1.geturl()  # 如果p_id不在实际url中，则说明发生了重定向，项目失败
            if self.isDirected:  
                rawhtml = f_1.read().decode('utf-8')
                self.h_soup = BeautifulSoup(rawhtml, 'html.parser')
                cat_div = self.h_soup.find('div', {'class': 'project-img'})
                l_cat = cat_div.i['class'][0]
                if l_cat == 'zc-orange-preheat':  # 预热中、众筹中、众筹成功、项目成功
                    self.category = '预热中'
                elif l_cat == 'zc-green-ing':
                    self.category = '众筹中'
                elif l_cat == 'zc-success':
                    self.category = '众筹成功'
                elif l_cat == 'xm-success':
                    self.category = '项目成功'

                url_2 = 'http://sq.jr.jd.com/cm/getCount?'
                req_2 = request.Request(url_2)
                req_2.add_header('User-Agent', self.User_Agent)
                req_2.add_header('Referer', 'http://z.jd.com/project/details/%s.html' % self.p_id)
                req_2.add_header('Host', self.Host)
                post_list = [('_', '15242091922'),
                             ('callback', 'jQuery183000564758012421237_1524209188840'),
                             ('key', '1000'),
                             ('pin', ''),
                             ('systemId', self.p_id),
                             ('temp', '0.29549820811900180')]  # 关键在于systemID，即项目编号
                post_data = parse.urlencode(post_list)
                with request.urlopen(req_2, data=post_data.encode('utf-8')) as f_2:
                    self.j_soup = f_2.read().decode()

    def basic_data(self):
        # (1)项目信息
        div1 = self.h_soup.find_all('div', {'class': 'project-introduce'})[0]
        proj_name = div1.find_all('h1', {'class': 'p-title'})[0].string  # 项目名称

        div1_2 = div1.find_all('p', {'class': "p-target"})[0]
        time_span = div1_2.find_all('span', {'class': 'f_red'})[0].string.strip()  # 提取截止日期
        # time_span = int(time_span)
        target_fund = div1_2.find_all('span', {'class': 'f_red'})[1].get_text()[1:]  # 目标金额

        s = self.h_soup.find_all('div', {'class': "tab-share-l"})
        sort_name = s[0].a['data-attr']  # 项目所属类别

        # (2)发起人信息
        div2 = self.h_soup.find_all('div', {'class': "promoters-name"})[0]
        prom_href = div2.find_all('a')[0]['href']  # 发起人href
        prom_name = div2.find_all('a')[0]['title']  # 发起人名称

        # (3)公司信息
        company_name, company_address, company_phone, company_hours = None, None, None, None  # 先设为None
        try:
            div3 = self.h_soup.find_all('ul', {'class': "contact-box"})[0]
            div3_li = div3.find_all('li')
            for li in div3_li:  # 少数项目没有公司名称和联系地址
                key = li.find('div', {'class': "key"}).contents[1]
                val = li.find('div', {'class': "val"}).string
                if key == '公司名称：':
                    company_name = val  # 公司名称
                elif key == '联系地址：':
                    company_address = val  # 联系地址
                elif key == '官方电话：':
                    company_phone = val  # 官方电话
                elif key == '工作时间：':
                    company_hours = val  # 工作时间
        except IndexError:
            print('No company info')

        # (4)各档支持信息
        div4 = self.h_soup.find_all('div', {'class': "box-grade "})
        indiv_info = {}
        for i, d in enumerate(div4):
            sup_price = d.find_all('div', {'class': "t-price"})[0].span.string.strip()  # 支持价位
            d_1 = d.find_all('div', {'class': "box-content"})
            lim_num = d_1[0].find_all('span', {'class': "limit-num"})[0].string  # 限制人数
            redound_info = d_1[0].find_all('p', {'class': "box-intro"})[0].string.strip()  # 回报内容
            deliver_info = d_1[0].find_all('p', {'class': "box-item"})[1].get_text()  # 发货时间
            indiv_info[str(i)] = {'sup_price': int(sup_price),
                                  'lim_num': lim_num,
                                  'redound_info': redound_info,
                                  'deliver_info': deliver_info}

        return {'项目名称': proj_name, '目标金额': int(target_fund), '众筹期限': time_span,
                '所属类别': sort_name, '发起人链接': prom_href, '发起人名称': prom_name,
                '公司名称': company_name, '公司地址': company_address, '公司工作时间': company_hours,
                '公司电话': company_phone, '各档基础信息': indiv_info}

    def update_data(self):
        # 当前筹集金额、当前进度、当前支持人数
        div1 = self.h_soup.find_all('div', {'class': 'project-introduce'})[0]
        try:
            div1_1 = div1.find_all('p', {'class': "p-progress"})[0]
            now_percent = div1_1.find_all('span', {'class': "fl percent"})[0].string[4:-1]  # 当前进度
            now_supporters = div1_1.find_all('span', {'class': "fr"})[0].string[:-4]  # 当前支持人数
            now_fund = div1.find_all('p', {'class': "p-num"})[0].get_text()[1:]  # 当前项目筹集金额
        except IndexError:
            now_percent, now_supporters, now_fund = 0, 0, 0
            print('  项目还未开始众筹！')

        if self.category == '众筹中':  # 获取end_time以决定何时获取review信息
            div1 = self.h_soup.find_all('div', {'class': 'project-introduce'})[0]
            div1_2 = div1.find_all('p', {'class': "p-target"})[0]
            end_time = div1_2.find_all('span', {'class': 'f_red'})[0].string.strip()  # 提取截止日期
            end_time = '-'.join(self.p_d.findall(end_time))
            self.end_time = datetime.datetime.strptime(end_time, '%Y-%m-%d')  # 转化截止日期为标准格式

        # 各档支持信息
        div4 = self.h_soup.find_all('div', {'class': "box-grade "})
        indiv_info = {'爬取时间': self.craw_time}
        for i, d in enumerate(div4):
            now_num_sup = d.find_all('div', {'class': "t-people"})[0].span.string.strip()  # 当前支持人数
            indiv_info[str(i)] = {'now_num_sup': int(now_num_sup)}

        # 当前点赞人数、当前关注者数、项目创建时间、项目更新时间
        p_praise = re.compile('"praise":(\d+)')
        p_focus = re.compile('"focus":(\d+)')
        p_createTime = re.compile('"createTime":"(\d+-\d+-\d+ \d+:\d+:\d+)"')
        p_updateTime = re.compile('"updateTime":"(\d+-\d+-\d+ \d+:\d+:\d+)"')

        praise = p_praise.findall(self.j_soup)[0]  # 点赞人数
        focus = p_focus.findall(self.j_soup)[0]  # 关注人数
        try:
            createTime = p_createTime.findall(self.j_soup)[0]
            createTime = datetime.datetime.strptime(createTime, "%Y-%m-%d %H:%M:%S") # 项目创建时间

            updateTime = p_updateTime.findall(self.j_soup)[0]  # 项目更新时间
            updateTime = datetime.datetime.strptime(updateTime, "%Y-%m-%d %H:%M:%S") # 项目创建时间
        except IndexError:
            createTime = datetime.datetime.now()
            updateTime = datetime.datetime.now()
            print('本项目还没有任何点赞和关注！')

        return {'项目动态信息': {'爬取时间': self.craw_time,
                                '筹集金额': int(now_fund),
                                '完成百分比': float(now_percent),
                                '支持者数': int(now_supporters),
                                '点赞数': int(praise),
                                '关注数': int(focus),
                                '创建时间': createTime,
                                '更新时间': updateTime},
                 '各档动态信息': indiv_info}

    def review_data(self):
        url = 'https://sq.jr.jd.com/topic/getTopicList?'
        req = request.Request(url)
        req.add_header('User-Agent', self.User_Agent)
        req.add_header('Referer', 'http://z.jd.com/project/details/%s.html' % self.p_id)
        req.add_header('Host', 'sq.jr.jd.com')
        reviews = {}
        i = 1
        while True:
            post_list = [('_', '1524311286832'),
                         ('callback', 'jQuery183036846271396508157_1524311279519'),
                         ('key', '1000'),
                         ('pageNo', '%d' % i),
                         ('pageSize', '20'),
                         ('serviceType', '1'),
                         ('sort', '1'),
                         ('systemId', '%s' % self.p_id),
                         ('temp', '0.8972247674942638')] # 关键在于systemID，即项目编号
            post_data = parse.urlencode(post_list)
            f = request.urlopen(req, data=post_data.encode('utf-8'))
            x = f.read().decode()
            json_x = json.loads(x[x.find('{"listResult"'):-1])
            pageBean = json_x['pageBean']
            totalPage = int(pageBean['totalPage'])  # 总页数
            totalRecord = int(pageBean['totalRecord'])  # 总记录数
            # 评论id, 创建时间，点赞数，回复数，内容
            for record in json_x['listResult']:
                reviews[str(record['topicId'])] = record  # 与原有数据库中不同，保留所有的评论详细信息
                
            if totalPage > i:
                i += 1
            else:
                break

        return {'爬取时间': self.craw_time, '总页数': totalPage, '总评论数': totalRecord, '评论详细': reviews}

    def start_craw(self):
        x = self.basic_data()
        y = self.update_data()
        x.update(y)
        return x

- 评论中未包含评论详细信息，因此重新爬取评论数据

In [ ]:
project.find_one({'状态': '众筹成功'})['评论']

In [ ]:
i = 1
ids = [x['_id'] for x in project.find({'状态': {'$in': ['众筹成功', '项目成功']}}, projection={'_id':1})]
print(f"一共{len(ids)}个文档需要更新")
t1 = time.process_time()
for p_id in ids:
    time.sleep(random.random())
    single_craw = Single_proj_craw(p_id)
    if i % 30 != 0:
        print(f"{i:3}", end=' ')
    else:
        print(f"{i:3}")
    try:
        x = single_craw.review_data()
        project.update_one({'_id':p_id}, {'$set': {'评论': x}}, upsert=True)
    except Exception as e:
        print(p_id, '出错!')
    i += 1
    
print(f'一共用时{(time.process_time() - t1)/60 : .2f}分钟')

- 先查看p_id是否在数据库中，如果在则直接获取数据；如果不在，则爬取数据。

In [5]:
col_list = ['项目名称', '目标金额', '众筹期限', '所属类别', '发起人链接', 
             '发起人名称', '公司名称', '公司地址', '公司工作时间', '公司电话', 
             '项目动态信息']

In [6]:
def checkOtherCols():
        suc_ids = list(set(x['_id'] for x in project.find({}, projection={'_id':1})))  # 除失败项目之外的所有项目信息
        for p_id in suc_ids[::-1]:  # 有部分文档以ObjectId作为_id
            if isinstance(p_id, ObjectId):
                print(f"success {p_id}")
                t_id = project.find_one({"_id":p_id}, projection={'项目编号':1})
                suc_ids.remove(p_id)
                suc_ids.append(t_id['项目编号'])
        
        fail_ids = list(set(x['详细信息']['_id'] for x in f_project.find({}, projection={'详细信息._id':1})))
        for p_id in fail_ids[::-1]:  # 将_id为ObjectID类型的替换为项目编号
            if isinstance(p_id, ObjectId):
                print(f"failure {p_id}")
                t_id = f_project.find_one({"详细信息._id":p_id}, projection={"详细信息.项目编号":1})
                fail_ids.remove(p_id)
                fail_ids.append(t_id["详细信息"]["项目编号"])
                
        return set(suc_ids) | set(fail_ids)

In [7]:
exist_ids = checkOtherCols()
ids = [x['_id'] for x in p_founder.find({}, projection={'_id':1})]
fail_list = np.load('fail_list.npy')

success 5af292a638ea07432da62d91
success 5af292a738ea07432da62d9a
success 5af292a738ea07432da62d92
success 5af292a738ea07432da62d97
success 5af292a638ea07432da62d90
success 5af292a738ea07432da62d93
success 5af292a738ea07432da62d94
success 5af292a738ea07432da62d98
success 5af292a738ea07432da62d99
success 5af292a738ea07432da62d95
success 5aeb00d458abe024f8d01956


In [10]:
fail_list = list(fail_list)

In [14]:
i = 4176
t1 = time.process_time()
for p_id in ids[4176:]:
    if p_id in exist_ids:
        info_dict = project.find_one({'_id':p_id}, projection={x:1 for x in col_list})
        if not info_dict:
            temp = f_project.find_one({'项目编号':p_id}, projection={'详细信息':1})
            info_dict = {x:temp['详细信息'][x] for x in temp['详细信息'] if x in col_list}
    else:
        try:
            single_craw = Single_proj_craw(p_id)
            temp = single_craw.start_craw()
            info_dict = dict([x for x in temp.items() if x[0] in col_list])
        except Exception as e:
            info_dict = {}
        
        if i % 10 != 0:
            print(f"{p_id:5} c", end='  ')
        else:
            print(f"{p_id:5} c")
    try:
        p_founder.update_one({'_id':p_id}, {'$set':info_dict}, upsert=True)
    except Exception as e:
        print(i, p_id, e)
        fail_list.append(p_id)
        
    i += 1
    
print(f'一共用时{(time.process_time() - t1)/60 : .2f}分钟！')

78321 c  No company info
43047 c  73667 c  No company info
30801 c  No company info
12896 c
60749 c  52778 c  4182 52778 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
No company info
13834 c  74383 c  88979 c  4185 88979 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
16158 c  4186 16158 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
91702 c  76292 c  76248 c  90489 c
4190 90489 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
71011 c  68931 c  3628  c  98603 c  66364 c  78181 c  No company info
19896 c  No company info
44150 c  77824 c  No company info
56084 c
70833 c  4201 70833 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
87837 c  4202 87837 '$set' is empty. You must specify a field like so: {$set: {<field>: ...}}
No company info
32988 c  97612 c  98167 c  76016 c  80404 c  90466 c  No company info
57966 c  90353 c
67704 c  4211 67704 '$set' is empty

In [ ]:
temp = f_project.find_one({'项目编号':'105325'}, projection={'详细信息':1})
if temp:
    info_dict = {x:temp['详细信息'][x] for x in temp['详细信息'] if x in col_list}

In [ ]:
single_craw = Single_proj_craw('101120')
x = single_craw.start_craw()

In [ ]:
x['评论详细']['1951617']

In [16]:
fail_list[0]

'102211'